In [3]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
#training
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip=0
face_data = []
dataset_path = "./face_data/"
filename = input("Enter the name:")

while True:
    ret,frame = cap.read()
    
    if ret == False:
        continue
    
    grayframe = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(grayframe,1.3,5)
    if(len(faces) == 0):
        continue
    faces = sorted(faces,key = lambda f:f[2]*f[3])
    
    for face in faces[-1:]:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        offset=10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        skip+=1
        if skip%10 == 0:
            face_data.append(face_section)
            print(len(face_data))
    cv2.imshow("Video frame",frame)
    cv2.imshow("Face section",face_section)
    #cv2.imshow("Gray frame",grayframe)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

np.save(dataset_path+filename+'.npy',face_data)
print("Data Successfully save at "+dataset_path+filename+'.npy')

cap.release()
cv2.destroyAllWindows()

Enter the name:keanu
1
2
3
4
5
6
7
8
9
10
11
(11, 30000)
Data Successfully save at ./face_data/keanu.npy


In [4]:
#knn algorithm
def distance(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(train, test, k=5):
    dist = []

    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]
    
    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]

In [5]:
#storing
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip=0
dataset_path = "./face_data/"

face_data=[]
labels = []

class_id = 0
names = {}

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        print("Loaded: ",fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)
print(trainset)

Loaded:  paras.npy
Loaded:  keanu.npy
(30, 30000)
(30, 1)
[[200. 214. 221. ...  39.  39.   0.]
 [194. 208. 215. ...  32.  29.   0.]
 [198. 210. 217. ...  25.  29.   0.]
 ...
 [ 37.  26.  25. ...  12.  14.   1.]
 [ 36.  24.  25. ...  10.  13.   1.]
 [ 37.  25.  23. ...   8.  13.   1.]]


In [6]:
#testing
cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    if not ret:
        continue
    grayframe = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(grayframe,1.3,5)
    
    for face in faces:
        x,y,w,h = face
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        out = knn(trainset,face_section.flatten())
        
        pred_name = names[out]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    cv2.imshow("Faces",frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()